# Explore real data to help with simulating test data

In [2]:
import malariagen_data
import allel
import numpy as np
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_dark"

In [3]:
ag3 = malariagen_data.Ag3(
    "simplecache::gs://vo_agam_release",
    simplecache=dict(cache_storage="../gcs_cache"),
)
ag3

<MalariaGEN Ag3 API client>
Storage URL             : simplecache::gs://vo_agam_release
Data releases available : 3.0
Results cache           : None
Cohorts analysis        : 20230223
AIM analysis            : 20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 0.0.0
Client location         : unknown
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact data@malariagen.net. For API documentation see 
https://malariagen.github.io/vector-data/ag3/api.html

In [4]:
af1 = malariagen_data.Af1(
    "simplecache::gs://vo_afun_release",
    simplecache=dict(cache_storage="../gcs_cache"),
)
af1

<MalariaGEN Af1 API client>
Storage URL             : simplecache::gs://vo_afun_release
Data releases available : 1.0
Results cache           : None
Cohorts analysis        : 20221129
Site filters analysis   : dt_20200416
Software version        : malariagen_data 0.0.0
Client location         : unknown
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact data@malariagen.net.

## Haplotypes

In [6]:
ds_snps = ag3.snp_calls(region="3L", sample_sets="AG1000G-BF-A")
ds_snps

<xarray.Dataset>
Dimensions:                             (variants: 40758473, alleles: 4,
                                         samples: 181, ploidy: 2)
Coordinates:
    variant_position                    (variants) int32 dask.array<chunksize=(524288,), meta=np.ndarray>
    variant_contig                      (variants) uint8 dask.array<chunksize=(524288,), meta=np.ndarray>
    sample_id                           (samples) <U24 dask.array<chunksize=(181,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele                      (variants, alleles) |S1 dask.array<chunksize=(524288, 1), meta=np.ndarray>
    variant_filter_pass_gamb_colu_arab  (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    variant_filter_pass_gamb_colu       (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    variant_filter_pass_arab            (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    call_genotype                       (variants, samples, ploidy) int8 dask.array<chunksize=(300000, 50, 2), meta=np.ndarray>
    call_GQ                             (variants, samples) int16 dask.array<chunksize=(300000, 50), meta=np.ndarray>
    call_MQ                             (variants, samples) int16 dask.array<chunksize=(300000, 50), meta=np.ndarray>
    call_AD                             (variants, samples, alleles) int16 dask.array<chunksize=(300000, 50, 4), meta=np.ndarray>
    call_genotype_mask                  (variants, samples, ploidy) bool dask.array<chunksize=(300000, 50, 2), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

In [24]:
ds_haps = ag3.haplotypes(
    region="3L", sample_sets="AG1000G-BF-A", analysis="gamb_colu_arab"
)
ds_haps

<xarray.Dataset>
Dimensions:           (variants: 10319068, alleles: 2, samples: 181, ploidy: 2)
Coordinates:
    variant_position  (variants) int32 dask.array<chunksize=(262144,), meta=np.ndarray>
    variant_contig    (variants) uint8 dask.array<chunksize=(10319068,), meta=np.ndarray>
    sample_id         (samples) object dask.array<chunksize=(181,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele    (variants, alleles) |S1 dask.array<chunksize=(262144, 1), meta=np.ndarray>
    call_genotype     (variants, samples, ploidy) int8 dask.array<chunksize=(262144, 64, 2), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

In [25]:
ds_haps.dims["variants"] / ds_snps.dims["variants"]

0.2531760206031271

In [26]:
ds_haps = ag3.haplotypes(region="3L", sample_sets="AG1000G-BF-A", analysis="gamb_colu")
ds_haps.dims["variants"] / ds_snps.dims["variants"]

0.27856833596292974

In [28]:
ds_haps = ag3.haplotypes(region="3L", sample_sets="AG1000G-UG", analysis="arab")
ds_haps.dims["variants"] / ds_snps.dims["variants"]

0.09217013601073819

In [8]:
snp_alleles = ds_snps["variant_allele"].values
snp_alleles

array([[b'A', b'C', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'G', b'A', b'C', b'T'],
       ...,
       [b'T', b'A', b'C', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'C', b'A', b'T', b'G']], dtype='|S1')

In [9]:
snp_pos = ds_snps["variant_position"].values
snp_pos

array([       1,        2,        3, ..., 41963433, 41963434, 41963435],
      dtype=int32)

In [10]:
loc_hap_sites = np.random.choice([False, True], size=snp_pos.shape[0], p=[0.75, 0.25])
loc_hap_sites

array([False, False,  True, ..., False,  True, False])

In [11]:
n_hap_sites = np.sum(loc_hap_sites)
n_hap_sites

10192236

In [12]:
sim_hap_ref = snp_alleles[loc_hap_sites, 0]
sim_hap_ref

array([b'G', b'A', b'G', ..., b'C', b'G', b'C'], dtype='|S1')

In [13]:
np.arange(1, 4)

array([1, 2, 3])

In [14]:
sim_alt_choice = np.random.choice(np.arange(1, 4), size=n_hap_sites)
sim_alt_choice

array([2, 3, 2, ..., 2, 3, 1])

In [15]:
sim_hap_alt = np.take_along_axis(
    snp_alleles[loc_hap_sites], indices=sim_alt_choice[:, None], axis=1
)[:, 0]
sim_hap_alt

array([b'C', b'G', b'C', ..., b'T', b'T', b'A'], dtype='|S1')

In [16]:
sim_hap_alt.shape

(10192236,)

In [17]:
ds_haps

<xarray.Dataset>
Dimensions:           (variants: 10319068, alleles: 2, samples: 181, ploidy: 2)
Coordinates:
    variant_position  (variants) int32 dask.array<chunksize=(262144,), meta=np.ndarray>
    variant_contig    (variants) uint8 dask.array<chunksize=(10319068,), meta=np.ndarray>
    sample_id         (samples) object dask.array<chunksize=(181,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele    (variants, alleles) |S1 dask.array<chunksize=(262144, 1), meta=np.ndarray>
    call_genotype     (variants, samples, ploidy) int8 dask.array<chunksize=(262144, 64, 2), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

In [29]:
ds_haps = ag3.haplotypes(
    region="3L", sample_sets="AG1000G-BF-A", analysis="gamb_colu_arab"
)
gt = ds_haps["call_genotype"][:1_000_000].values
p_hap_01 = np.bincount(gt.flatten())
p_hap_01 = p_hap_01 / p_hap_01.sum()
p_hap_01

array([0.9922709, 0.0077291])

In [30]:
ds_haps = ag3.haplotypes(region="3L", sample_sets="AG1000G-BF-A", analysis="gamb_colu")
gt = ds_haps["call_genotype"][:1_000_000].values
p_hap_01 = np.bincount(gt.flatten())
p_hap_01 = p_hap_01 / p_hap_01.sum()
p_hap_01

array([0.99058904, 0.00941096])

In [31]:
ds_haps = ag3.haplotypes(region="3L", sample_sets="AG1000G-UG", analysis="arab")
gt = ds_haps["call_genotype"][:1_000_000].values
p_hap_01 = np.bincount(gt.flatten())
p_hap_01 = p_hap_01 / p_hap_01.sum()
p_hap_01

array([0.94134709, 0.05865291])

In [21]:
hap_gt_sim = np.random.choice(
    [0, 1], size=(100_000, 100), replace=True, p=p_hap_01
).astype("i1")
hap_gt_sim

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [23]:
np.bincount(hap_gt_sim.flatten())

array([9922335,   77665])

## Site annotations

In [ ]:
ann = ag3.open_site_annotations()
ann

In [ ]:
list(ann)

In [ ]:
print(ann.tree())

In [ ]:
codon_degeneracy = ann["codon_degeneracy"]["3L"][10_000_000:11_000_000]
px.histogram(codon_degeneracy)

In [ ]:
np.bincount(codon_degeneracy + 1) / codon_degeneracy.size

In [ ]:
codon_nonsyn = ann["codon_nonsyn"]["3L"][10_000_000:11_000_000]
px.histogram(codon_nonsyn)

In [ ]:
np.bincount(codon_nonsyn) / codon_nonsyn.size

In [ ]:
codon_position = ann["codon_position"]["3L"][10_000_000:11_000_000]
px.histogram(codon_position)

In [ ]:
np.bincount(codon_position + 1) / codon_position.size

In [ ]:
seq_cls = ann["seq_cls"]["3L"][10_000_000:11_000_000]
px.histogram(seq_cls)

In [ ]:
np.bincount(seq_cls) / seq_cls.size

In [ ]:
seq_flen = ann["seq_flen"]["3L"][10_000_000:11_000_000]
px.histogram(seq_flen, nbins=20)

In [ ]:
seq_relpos_start = ann["seq_relpos_start"]["3L"][10_000_000:11_000_000]
px.histogram(seq_relpos_start, nbins=20)

In [ ]:
seq_relpos_stop = ann["seq_relpos_stop"]["3L"][10_000_000:11_000_000]
px.histogram(seq_relpos_stop, nbins=20)

In [ ]:
x = np.random.beta(a=0.4, b=4, size=100_000) * 40_000
px.histogram(x, nbins=20)

In [ ]:
list(ann)

## SNP calls

In [ ]:
ds = ag3.snp_calls(region="3L", sample_sets="AG1000G-BF-A")
ds

In [ ]:
ds2 = ds.isel(variants=slice(10_500_000, 10_600_000))
ds2

In [ ]:
gt = ds2["call_genotype"].values
gt

In [ ]:
gt2 = allel.GenotypeArray(gt)
gt2

In [ ]:
np.bincount(gt.flatten() + 1) / gt.size

In [ ]:
missing_fraction = gt2.count_missing() / (gt2.n_variants * gt2.n_samples)
missing_fraction

In [ ]:
allele_counts = np.bincount(gt.flatten() + 1)[1:]
allele_counts

In [ ]:
allele_fractions = allele_counts / np.sum(allele_counts)
allele_fractions

In [ ]:
gt_sim = np.random.choice(4, size=gt.shape, replace=True, p=allele_fractions)
gt_sim

In [ ]:
n_calls = gt_sim.shape[0] * gt_sim.shape[1]
n_calls

In [ ]:
gt_sim_flat = gt_sim.reshape(-1, 2)
gt_sim_flat

In [ ]:
loc_missing = np.random.choice(
    n_calls,
    size=int(missing_fraction * n_calls),
    replace=False,
)
loc_missing

In [ ]:
gt_sim_flat[loc_missing] = -1

In [ ]:
gt_sim

In [ ]:
np.bincount(gt_sim.flatten() + 1) / gt_sim.size

In [ ]:
np.bincount(gt.flatten() + 1) / gt.size

In [ ]:
allel.GenotypeArray(gt).count_missing()

In [ ]:
allel.GenotypeArray(gt_sim).count_missing()

In [ ]:
allel.GenotypeArray(gt).count_hom_ref()

In [ ]:
allel.GenotypeArray(gt_sim).count_hom_ref()

In [ ]:
allel.GenotypeArray(gt).count_hom_alt()

In [ ]:
allel.GenotypeArray(gt_sim).count_hom_alt()

In [ ]:
gq = ds2["call_GQ"].values

In [ ]:
px.histogram(gq.flatten()[:100_000])

In [ ]:
gqf = gq.flatten()
n_gq = np.bincount(gqf[gqf >= 0])
n_gq

In [ ]:
p_gq = np.bincount(gqf[gqf >= 0]) / gqf.size
p_gq

In [ ]:
px.bar(p_gq)

In [ ]:
mq = ds2["call_MQ"].values
mq

In [ ]:
mqf = mq.flatten()

In [ ]:
px.histogram(mqf[:100_000])

In [ ]:
p_mq = np.bincount(mqf[mqf >= 0]) / mqf.size
p_mq

In [ ]:
px.bar(p_mq)

In [ ]:
ad = ds2["call_AD"].values
ad

In [ ]:
ad0 = ad[:, :, 0].flatten()
ad0

In [ ]:
ad1 = ad[:, :, 1].flatten()
ad1

In [ ]:
ad2 = ad[:, :, 2].flatten()
ad3 = ad[:, :, 3].flatten()

In [ ]:
px.histogram(ad0[:10_000])

In [ ]:
px.histogram(ad1[:10_000])

In [ ]:
px.histogram(ad2[:10_000])

In [ ]:
p_ad0 = np.bincount(ad0[ad0 >= 0]) / ad0.size
px.bar(p_ad0)

In [ ]:
p_ad1 = np.bincount(ad1[ad1 >= 2]) / ad1.size
px.bar(p_ad1)

In [ ]:
p_ad2 = np.bincount(ad2[ad2 >= 2]) / ad1.size
px.bar(p_ad2)

In [ ]:
p_ad3 = np.bincount(ad3[ad3 >= 2]) / ad1.size
px.bar(p_ad3)

In [ ]:
pos = ds2["variant_position"].values

In [ ]:
pos

In [ ]:
px.line(pos[:100_000])

In [ ]:
alleles = ds2["variant_allele"].values[:10]
alleles

In [ ]:
ref = alleles[:, 0]

In [ ]:
alt_sim = np.empty(shape=(alleles.shape[0], alleles.shape[1] - 1), dtype="S1")
alt_sim[ref == b"A"] = np.array([b"C", b"T", b"G"])
alt_sim[ref == b"C"] = np.array([b"A", b"T", b"G"])
alt_sim[ref == b"T"] = np.array([b"A", b"C", b"G"])
alt_sim[ref == b"G"] = np.array([b"A", b"C", b"T"])
alt_sim

In [ ]:
ds2

In [ ]:
pass_gc = ds2["variant_filter_pass_gamb_colu"].values
p_pass_gc = np.sum(pass_gc) / pass_gc.size
p_pass_gc

In [ ]:
pass_a = ds2["variant_filter_pass_arab"].values
p_pass_a = np.sum(pass_a) / pass_a.size
p_pass_a

In [ ]:
pass_gca = ds2["variant_filter_pass_gamb_colu_arab"].values
p_pass_gca = np.sum(pass_gca) / pass_gca.size
p_pass_gca

## Sequence composition

In [ ]:
ag_seq = ag3.genome_sequence("3L").compute()
ag_seq

In [ ]:
from collections import Counter

In [ ]:
ag_seq_count = Counter(ag_seq)
ag_seq_count

In [ ]:
filter_pass = af1.site_filters(region="3RL:10,000,000-11,000,000", mask="funestus")
filter_pass

In [ ]:
np.sum(filter_pass).compute() / filter_pass.size

In [ ]:
af_seq = af1.genome_sequence("3RL")[:60_000_000].compute()
af_seq

In [ ]:
bases = np.array([b"a", b"c", b"g", b"t", b"n", b"A", b"C", b"G", b"T", b"N"])
bases

In [ ]:
af_seq_count = Counter(af_seq)
af_seq_count

In [ ]:
p_bases_af = {b: af_seq_count[b] / af_seq.size for b in bases}
p_bases_af

In [ ]:
p_ad0 = np.bincount(ad0[ad0 >= 0]) / ad0.size
px.bar(p_ad0)

In [ ]:
p_bases_ag = {b: ag_seq_count[b] / ag_seq.size for b in bases}
p_bases_ag

In [ ]:
p_bases_ag = np.array([ag_seq_count[b] for b in bases]) / ag_seq.size
p_bases_ag

In [ ]:
p_bases_ag.sum()

In [ ]:
seq_sim = np.random.choice(bases, size=ag_seq.size, replace=True, p=p_bases_ag)
seq_sim

In [ ]:
af1.sample_sets()

In [ ]:
ds = af1.snp_calls(
    region="3RL:10_000_000-10_500_000", sample_sets="1230-VO-GA-CF-AYALA-VMF00045"
)
ds

In [ ]:
gt = ds["call_genotype"].values
gt

In [ ]:
allele_counts = np.bincount(gt.flatten() + 1)[1:]
allele_counts

In [ ]:
allele_counts / np.sum(allele_counts)

In [ ]:
np.sum(gt < 0) / gt.size